# This notebook is for Clusterting Assignment

In [2]:
from bs4 import BeautifulSoup
import requests
import folium
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
 

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Age': '74482', 'Content-language': 'en', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'X-Cache-Status': 'hit-front', 'X-Content-Type-Options': 'nosniff', 'Backend-Timing': 'D=113251 t=1555940901776477', 'Accept-Ranges': 'bytes', 'Last-Modified': 'Sat, 20 Apr 2019 14:00:51 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Server': 'mw1321.eqiad.wmnet', 'X-Client-IP': '158.176.106.111', 'X-Analytics': 'ns=0;page_id=539066;https=1;nocookies=1', 'Content-Length': '15001', 'P3P': 'CP="This is not a P3P policy! See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-Encoding': 'gzip', 'Set-Cookie': 'WMF-Last-Access=23-Apr-2019;Path=/;HttpOnly;secure;Expires=Sat, 25 May 2019 00:00:00 GMT, WMF-Last-Access-Global=23-Apr-2019;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Sat, 25 May 2019 00:00:00 GMT, GeoIP=US:::37.75:-97.82:v4; Path=/; secure; Domain=.

In [4]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

# lst

In [5]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)
# df[df.duplicated(['PostalCode'], keep=False)] - this would have shown the duplicate PostalCodes

(211, 3)


In [6]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

In [7]:
df.loc[df['PostalCode'] == 'M5A'] ##checking for M5A

,PostalCode,Neighborhood,Borough
53,M5A,"Harbourfront, Regent Park",Downtown Toronto


In [10]:
df.shape  #Shape of dataframe

(103, 3)

# will Read GEO CSV DATA HERE

In [12]:
dfgeo = pd.read_csv("http://cocl.us/Geospatial_data")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

* will merge 2 dataframes based on a PostalCode column

In [14]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')


* will check for M5G

In [15]:
df2.loc[df2['PostalCode'] == 'M5G']

,PostalCode,Neighborhood,Borough,Latitude,Longitude
57,M5G,Central Bay Street,Downtown Toronto,43.657952,-79.387383


* looking at top 10 observations

In [19]:
df2.head(10)

,PostalCode,Neighborhood,Borough,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711
3,M1G,Woburn,Scarborough,43.770992,-79.216917
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476
5,M1J,Scarborough Village,Scarborough,43.744734,-79.239476
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough,43.727929,-79.262029
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough,43.711112,-79.284577
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough,43.716316,-79.239476
9,M1N,"Birch Cliff, Cliffside West",Scarborough,43.692657,-79.264848


* Installing Folium

In [32]:
import sys
!{sys.executable} -m pip install folium

    100% |████████████████████████████████| 92kB 6.0MB/s eta 0:00:01
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/

In [35]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.653963, -79.387207.


* Using Foursquare

In [36]:
CLIENT_ID = 'RKIUHGEAUGHFKHFYRCKFCX1L3TZISMUN1T2HCJ443C5G10WU' # Foursquare ID
CLIENT_SECRET = '15J43UABC0SCLENO5CDYQ2YBWXP4RXWZNQJSUSOIQDZ5E2MS' # Foursquare Secret
VERSION = '20190423' # API version

* Lets Focus on Etobicoke

In [37]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [38]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=RKIUHGEAUGHFKHFYRCKFCX1L3TZISMUN1T2HCJ443C5G10WU&client_secret=15J43UABC0SCLENO5CDYQ2YBWXP4RXWZNQJSUSOIQDZ5E2MS&v=20190423&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [39]:
results = requests.get(url).json()
# results

In [40]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [41]:
venues = results['response']['groups'][0]['items']

In [42]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4c633939e1621b8d48842553-0,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",4c633939e1621b8d48842553,6210 Finch Ave. W Suite 103,CA,Etobicoke,Canada,at Albion Rd.,344,"[6210 Finch Ave. W Suite 103 (at Albion Rd.), ...","[{'lat': 43.742421428383224, 'label': 'display...",43.742421,-79.589471,M9V 3Y5,ON,Subway,0,[]
1,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4b04a05bf964a520c45522e3-1,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",4b04a05bf964a520c45522e3,1530 Albion Rd,CA,Etobicoke,Canada,at Finch Ave. W.,318,"[1530 Albion Rd (at Finch Ave. W.), Etobicoke ...","[{'lat': 43.74196806166214, 'label': 'display'...",43.741968,-79.586639,M9V 1B4,ON,Sheriff's No Frills,0,[]
2,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4be58dc4cf200f479154133c-2,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",4be58dc4cf200f479154133c,1530 Albion Rd,CA,Etobicoke,Canada,Albion Mall,438,"[1530 Albion Rd (Albion Mall), Etobicoke ON M9...","[{'lat': 43.74083239297218, 'label': 'display'...",43.740832,-79.583347,M9V 1B4,ON,Shoppers Drug Mart,0,[]
3,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4be70e26cf200f47e334153c-3,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",4be70e26cf200f47e334153c,1530 Albion Rd.,CA,Etobicoke,Canada,at Kipling Ave. (Albion Centre),370,[1530 Albion Rd. (at Kipling Ave. (Albion Cent...,"[{'lat': 43.74120165509377, 'label': 'display'...",43.741202,-79.584545,M9V 1B4,ON,Popeyes Louisiana Kitchen,0,[]
4,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4cd4738cdfb4a1cd4337535c-4,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",4cd4738cdfb4a1cd4337535c,1530 Albion Rd,CA,Etobicoke,Canada,Near Finch Ave. W.,413,"[1530 Albion Rd (Near Finch Ave. W.), Etobicok...","[{'lat': 43.7416936, 'label': 'display', 'lng'...",43.741694,-79.584373,M9V 1B4,ON,The Beer Store,0,[]
5,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4d8ba6910c4e41bdaaf7667f-5,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",4d8ba6910c4e41bdaaf7667f,"1530 Albion Road, Unit T25",CA,Etobicoke,Canada,NaN,397,"[1530 Albion Road, Unit T25, Etobicoke ON M9V ...","[{'lat': 43.74156896801906, 'label': 'display'...",43.741569,-79.584489,M9V 1B4,ON,Pizza Pizza,0,[]
6,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-5112b872e4b0c0a78d7bcd27-6,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",5112b872e4b0c0a78d7bcd27,1620 Albion road,CA,Toronto,Canada,Albion Road and Finch Ave,319,"[1620 Albion road (Albion Road and Finch Ave),...","[{'lat': 43.74184023292111, 'label': 'display'...",43.741840,-79.590561,NaN,ON,Sunny Foodmart,0,[]
7,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4c1951d6834e2d7f2d3a2a80-7,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",4c1951d6834e2d7f2d3a2a80,1530 Albion Road,CA,Toronto,Canada,NaN,404,"[1530 Albion Road, Toronto ON M9V 1B4, Canada]","[{'lat': 43.74163461048035, 'label': 'display'...",43.741635,-79.584446,M9V 1B4,ON,McDonald's,0,[]


In [43]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Subway,Sandwich Place,43.742421,-79.589471
1,Sheriff's No Frills,Grocery Store,43.741968,-79.586639
2,Shoppers Drug Mart,Pharmacy,43.740832,-79.583347
3,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
4,The Beer Store,Beer Store,43.741694,-79.584373


In [44]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
df2

,Neighborhood,Borough,Latitude,Longitude
PostalCode,,,,
M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353
M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,43.784535,-79.160497
M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711
M1G,Woburn,Scarborough,43.770992,-79.216917
M1H,Cedarbrae,Scarborough,43.773136,-79.239476
M1J,Scarborough Village,Scarborough,43.744734,-79.239476
M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough,43.727929,-79.262029
M1L,"Clairlea, Golden Mile, Oakridge",Scarborough,43.711112,-79.284577
M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough,43.716316,-79.239476


In [47]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [48]:
print(venues.shape)
venues.head()

(2249, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Scarborough,43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,Scarborough,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [49]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,106,106,106,106,106,106
Downtown Toronto,1292,1292,1292,1292,1292,1292
East Toronto,123,123,123,123,123,123
East York,73,73,73,73,73,73
Etobicoke,74,74,74,74,74,74
Mississauga,11,11,11,11,11,11
North York,238,238,238,238,238,238
Queen's Park,42,42,42,42,42,42
Scarborough,90,90,90,90,90,90


In [50]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 279 unique categories.


In [51]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
onehot.shape

(2249, 279)

In [53]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.009434,0.000000,0.000000,0.009434,0.000000,0.000000,0.000000,0.000000,0.009434
1,Downtown Toronto,0.000000,0.000774,0.000774,0.000774,0.000774,0.000774,0.001548,0.002322,0.001548,...,0.002322,0.012384,0.001548,0.000000,0.003870,0.000000,0.006192,0.000774,0.000000,0.004644
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016260
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.013699,0.000000,0.013699,0.000000,0.000000,0.000000,0.013699
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004202,0.000000,0.000000,0.004202,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.004202,0.004202,0.008403,0.000000,0.000000,0.004202,0.008403,0.000000
7,Queen's Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.023810
8,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011050,0.000000,0.000000,0.011050,0.000000,0.005525,0.000000,0.000000,0.005525


In [54]:
grouped.shape

(11, 279)

In [55]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.08
2            Park  0.06
3            Café  0.05
4     Pizza Place  0.05


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2        Hotel  0.03
3       Bakery  0.03
4   Restaurant  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.07
2  Italian Restaurant  0.04
3      Ice Cream Shop  0.04
4                Café  0.04


----East York----
                 venue  freq
0          Coffee Shop  0.07
1  Sporting Goods Shop  0.04
2             Pharmacy  0.04
3          Pizza Place  0.04
4                 Park  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.11
1  Sandwich Place  0.07
2            Park  0.05
3        Pharmacy  0.05
4     Coffee Shop  0.04


----Mississauga----
                       venue  freq
0                Coffee Shop  0.18
1                      Hotel  0.18
2  

In [56]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [57]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Pizza Place,Café,Dessert Shop,Sushi Restaurant,Gym,Burger Joint,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Seafood Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Café,Ice Cream Shop,Italian Restaurant,Park,Brewery,American Restaurant,Bakery,Bookstore
3,East York,Coffee Shop,Pizza Place,Pharmacy,Bank,Park,Sporting Goods Shop,Burger Joint,Gym / Fitness Center,Grocery Store,Sandwich Place
4,Etobicoke,Pizza Place,Sandwich Place,Pharmacy,Park,Fast Food Restaurant,Gym,Coffee Shop,Grocery Store,Café,Pool
5,Mississauga,Hotel,Coffee Shop,Gym / Fitness Center,Mediterranean Restaurant,Middle Eastern Restaurant,Burrito Place,Sandwich Place,American Restaurant,Fried Chicken Joint,Dog Run
6,North York,Coffee Shop,Clothing Store,Fast Food Restaurant,Grocery Store,Pizza Place,Restaurant,Japanese Restaurant,Bank,Shopping Mall,Park
7,Queen's Park,Coffee Shop,Gym,Japanese Restaurant,Diner,Yoga Studio,College Auditorium,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place
8,Scarborough,Chinese Restaurant,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Coffee Shop,Indian Restaurant,Sandwich Place,Skating Rink,Pharmacy
9,West Toronto,Bar,Coffee Shop,Café,Italian Restaurant,Restaurant,Pizza Place,Bakery,Breakfast Spot,Diner,Park


In [58]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 0, 0, 3, 3, 1, 3, 4, 3, 0], dtype=int32)